- `sudo dmidecode -t baseboard`
    - 查看自己的主板型号以及 onboard device

```
Handle 0x0049, DMI type 2, 15 bytes
Base Board Information
	Manufacturer: H3C
	Product Name: BS56M2C20SAMDA
```

- lspci 看不到 cpu，查看 cpu 需要通过 lscpu
    -  CPU 不是插在 PCI/PCIe 插槽里的“外设”，恰恰相反，CPU 是整个系统的“主人”，而所有的 PCI/PCIe 插槽都是通过主板连接到 CPU 上的“仆人”。
    -  CPU 有自己专门的插槽（CPU socket）

In [2]:
from IPython.display import Image

In [3]:
# https://www.supermicro.com/zh_tw/products/motherboard/x12dpg-qt6
Image(url='https://www.supermicro.com/files_SYS/images/Motherboard/X12DPG-QT6.png', width=400)

- CPU插槽：图中有两个大块区域带有黄色警告标签，这是两个CPU插槽，用于安装服务器级的多核处理器。通常情况下，这种主板支持双路处理器。
- 内存插槽：这些蓝色和黑色的长条插槽是内存插槽（DIMM插槽）。这种主板一般支持大量的内存，以满足服务器和数据中心的高性能需求。
    - 每个CPU插槽周围有8个内存插槽。
- PCIe插槽：图中的多个黑色插槽是PCIe插槽，用于安装扩展卡，如显卡、网卡和存储控制器。
    - 1 PCIe 4.0 x8,
    - 6 PCIe 4.0 x16


### PCIe

- `sudo dmidecode -t slot`
    - Designation: 插槽的物理标识（如 PCI-E Slot 1）。
    - Type: 插槽类型（如 PCI Express）。
    - Current Usage: 插槽当前的使用状态（Available 或 In Use）。
        - In Use：说明这个插槽已经插了设备。
        - Available：说明这个插槽是空的，可以用来安装新硬件。
    - Length: 插槽的物理长度（如 Long）。

In [4]:
Image(url='./imgs/cpu-pcie.jpeg', width=400)

- 查看使用中的 PCIe 插槽外接的什么设备
    - 计算：`VGA compatible controller: NVIDIA Corporation Device 2684 (rev a1)`：4090
    - 存储
        - `RAID bus controller: Broadcom / LSI MegaRAID 12GSAS/PCIe Secure SAS39xx`
        - `Non-Volatile memory controller: Samsung Electronics Co Ltd NVMe SSD Controller PM9A1/PM9A3/980PRO`
    - 网络

```shell
sudo apt install gawk

sudo dmidecode -t slot | gawk -v RS= '
/Current Usage: In Use/ {
    if (match($0, /Designation: (.*)/, d) && match($0, /Bus Address: (.*)/, b)) {
        addr = b[1]
        if (addr != "0000:ff:00.0") {
            cmd = "lspci -s " addr
            cmd | getline device
            close(cmd)
            gsub(/^[0-9a-fA-F:.]+ /, "", device)
            printf "插槽 (Slot) \033[1;33m%s\033[0m (地址: %s) -> \033[1;32m%s\033[0m\n", d[1], addr, device
        }
    }
}'
```

### 内存条

- `sudo dmidecode -t memory`
- `sudo lshw -short -C memory`
    - 查看支持几个内存条及已经安装了几个内存条；